In [1]:
import os
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier,Pool,cv
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import metrics
import random 


In [2]:
curr_ym_num=201908
data_dir="/home/k2uxam/data/DU_data/"

In [3]:
from os import listdir

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [4]:
filenames = find_csv_filenames(data_dir)
for name in filenames:
  print(name)

file_sort_name=sorted(filenames)
print(file_sort_name)

du_apps_201808.csv
du_apps_201904.csv
du_apps_201810.csv
du_apps_201905.csv
du_apps_201901.csv
du_apps_201812.csv
du_apps_201907.csv
du_apps_201903.csv
du_apps_201811.csv
du_apps_201809.csv
du_apps_201906.csv
du_apps_201908.csv
du_apps_201902.csv
['du_apps_201808.csv', 'du_apps_201809.csv', 'du_apps_201810.csv', 'du_apps_201811.csv', 'du_apps_201812.csv', 'du_apps_201901.csv', 'du_apps_201902.csv', 'du_apps_201903.csv', 'du_apps_201904.csv', 'du_apps_201905.csv', 'du_apps_201906.csv', 'du_apps_201907.csv', 'du_apps_201908.csv']


In [5]:
du_files=list() #10:19-

In [6]:
for i in range(0,12):
    monthly_file=pd.read_csv((data_dir+file_sort_name[i]),sep='|')

    du_files.insert(i,monthly_file)

/home/k2uxam/mypython/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,37,67,69,71,73,76,78,79,91,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/k2uxam/mypython/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,67,69,71,73,76,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/k2uxam/mypython/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,4,36,37,67,69,71,73,76,78,79,91,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/k2uxam/mypython/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,37,67,69,71,73,76,78,79,91) have mixed

In [7]:
duapp_df=pd.concat(du_files)


In [8]:
duapp_df_clean = duapp_df[(duapp_df['PREQUAL_FLG'] != 1) & (duapp_df['LAST_CASEFILE_INSTID'] == 1) & 
         (~(duapp_df['LN_RPTG_TYP_DESC'].str.contains('Federal Housing Administration') | duapp_df['LN_RPTG_TYP_DESC'].str.contains('Veterans Affairs'))) &
        (duapp_df['REFI_PLUS_IND'] == 0 ) & (duapp_df['CASE_FICO_NO'] >=620) ]

In [9]:
duapp_df_clean.dropna(subset=['BRWR_TOT_DTI_RTO_PCT'])
duapp_df_clean['ACQ_FLAG'] = np.where(duapp_df_clean.FNMA_LN.isna(),0,1)

/home/k2uxam/mypython/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
inSample_acq=duapp_df_clean[(duapp_df_clean['ACQ_FLAG'] == 1)]
inSample_acq.shape

(1533950, 100)

In [11]:
inSample_unacq=duapp_df_clean[(duapp_df_clean['ACQ_FLAG'] == 0)]
inSample_unacq.shape

(4496370, 100)

In [12]:
inSample_unacq_B= inSample_unacq.sample(inSample_acq.shape[0])
inSample_unacq_B.shape

(1533950, 100)

In [13]:
inSample_data= inSample_acq.append(inSample_unacq_B)
train_data = inSample_data[["GROSMTHLYINCMAMT","BRWR_TOT_DTI_RTO_PCT","LN_APPL_INT_RT","LN_CLCD_LN_TO_VAL_RTO_PCT","PROP_APPRD_VAL_AMT","UWRG_LN_APPL_AMT","SUBCOUNT","UPBDIFF","CASE_FICO_NO","LN_COMB_LN_TO_VAL_RTO_PCT","LN_MTRY_TERM_MTHS_NO","LN_BRWR_CNT","NOMTHSRESVS","MTHLYHSNGEXP","TOTMNTHLYHSNGEXP","ELGCHG","TRIAL_LENDER","DU_ELGB_IND","PIW_OFR_IND","FRSTTMHMBYRIND","HIGHBAL","LN_AMRT_TYP_CD","LN_GUAR_TYP_CD","LN_PURPS_TYP_DESC","PRODFLG","PROP_USG_TYP_ID","PROP_CATG_TYP_ID","LN_RPTG_TYP_DESC"]]
train_data[1:5]

,GROSMTHLYINCMAMT,BRWR_TOT_DTI_RTO_PCT,LN_APPL_INT_RT,LN_CLCD_LN_TO_VAL_RTO_PCT,PROP_APPRD_VAL_AMT,UWRG_LN_APPL_AMT,SUBCOUNT,UPBDIFF,CASE_FICO_NO,LN_COMB_LN_TO_VAL_RTO_PCT,...,PIW_OFR_IND,FRSTTMHMBYRIND,HIGHBAL,LN_AMRT_TYP_CD,LN_GUAR_TYP_CD,LN_PURPS_TYP_DESC,PRODFLG,PROP_USG_TYP_ID,PROP_CATG_TYP_ID,LN_RPTG_TYP_DESC
18,8947.74,31.289,4.125,69.0,525000.0,360000.0,12.0,0.0,731,69.0,...,0,1,0,1,1,Purchase,1,1,9,Conforming
19,11833.33,29.112,5.500,92.0,501000.0,442000.0,7.0,-3500.0,660,92.0,...,0,1,0,1,1,Purchase,1,1,4,Conforming
20,8333.34,28.662,4.250,80.0,385000.0,296812.0,5.0,0.0,723,80.0,...,0,1,0,1,1,Purchase,1,1,4,Conforming
21,7291.67,30.126,4.875,90.0,214000.0,189000.0,5.0,0.0,766,90.0,...,0,1,0,1,1,Purchase,1,1,3,Conforming


In [14]:
label = inSample_data.ACQ_FLAG
#label

In [15]:
w=1.00/inSample_data.shape[0]
weight = np.repeat(w,inSample_data.shape[0])
weight

array([3.25955866e-07, 3.25955866e-07, 3.25955866e-07, ...,
       3.25955866e-07, 3.25955866e-07, 3.25955866e-07])

In [16]:

cat_features=range(15,28)
cat_features

[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

In [17]:

model=CatBoostClassifier(iterations=50,depth=6,task_type="GPU",devices='0:1')
model.fit(train_data,label,cat_features=cat_features)#sample_weight default =1

Learning rate set to 0.5
0:	learn: 0.5912126	total: 22.6ms	remaining: 1.11s
1:	learn: 0.5600727	total: 43.4ms	remaining: 1.04s
2:	learn: 0.5456629	total: 64.2ms	remaining: 1s
3:	learn: 0.5392600	total: 85.3ms	remaining: 981ms
4:	learn: 0.5360730	total: 106ms	remaining: 957ms
5:	learn: 0.5326340	total: 127ms	remaining: 935ms
6:	learn: 0.5303328	total: 148ms	remaining: 911ms
7:	learn: 0.5285673	total: 170ms	remaining: 891ms
8:	learn: 0.5258107	total: 191ms	remaining: 869ms
9:	learn: 0.5247003	total: 211ms	remaining: 844ms
10:	learn: 0.5238763	total: 232ms	remaining: 822ms
11:	learn: 0.5227679	total: 252ms	remaining: 799ms
12:	learn: 0.5218708	total: 272ms	remaining: 776ms
13:	learn: 0.5212188	total: 293ms	remaining: 753ms
14:	learn: 0.5202294	total: 314ms	remaining: 732ms
15:	learn: 0.5191645	total: 334ms	remaining: 709ms
16:	learn: 0.5186497	total: 354ms	remaining: 688ms
17:	learn: 0.5182839	total: 374ms	remaining: 665ms
18:	learn: 0.5176639	total: 394ms	remaining: 643ms
19:	learn: 0.51

#cv and out-of-sample
cv_dataset=Pool(data=train_data,
               label=label,
               cat_features=cat_features)
params = {"iterations": 100,
          "depth": 5,
          "loss_function": "Logloss",
          "roc_file":"roc-file"
         }
scores = cv(cv_dataset,
            params,
            fold_count=10, 
            plot="True")
#save in_sample Results

In [18]:
inSample_unbal=duapp_df_clean[["GROSMTHLYINCMAMT","BRWR_TOT_DTI_RTO_PCT","LN_APPL_INT_RT","LN_CLCD_LN_TO_VAL_RTO_PCT","PROP_APPRD_VAL_AMT","UWRG_LN_APPL_AMT","SUBCOUNT","UPBDIFF","CASE_FICO_NO","LN_COMB_LN_TO_VAL_RTO_PCT","LN_MTRY_TERM_MTHS_NO","LN_BRWR_CNT","NOMTHSRESVS","MTHLYHSNGEXP","TOTMNTHLYHSNGEXP","ELGCHG","TRIAL_LENDER","DU_ELGB_IND","PIW_OFR_IND","FRSTTMHMBYRIND","HIGHBAL","LN_AMRT_TYP_CD","LN_GUAR_TYP_CD","LN_PURPS_TYP_DESC","PRODFLG","PROP_USG_TYP_ID","PROP_CATG_TYP_ID","LN_RPTG_TYP_DESC","ACQ_FLAG"]].dropna()
du_pred_prob=model.predict_proba(inSample_unbal)


In [19]:
list_inSample_res=[]
list_inSample_res.append(inSample_unbal)
list_inSample_res.append(du_pred_prob)

In [20]:
survival_ratio=[x[1] for x in du_pred_prob]

In [21]:
auc_score_inSample = metrics.roc_auc_score(inSample_unbal.ACQ_FLAG,survival_ratio)
auc_score_inSample

0.8169024628847207

In [22]:
#import one-month outofsample DU and test
outSample_df=pd.read_csv('/home/k2uxam/data/DU_data/du_apps_201908.csv',sep='|')
outSample_df = outSample_df[(outSample_df['LAST_CASEFILE_INSTID'] == 1) & 
         (~(outSample_df['LN_RPTG_TYP_DESC'].str.contains('Federal Housing Administration') | outSample_df['LN_RPTG_TYP_DESC'].str.contains('Veterans Affairs'))) &
        (outSample_df['REFI_PLUS_IND'] == 0 ) & (outSample_df['CASE_FICO_NO'] >=620) ]
outSample_df.dropna(subset=['BRWR_TOT_DTI_RTO_PCT'])
outSample_df['ACQ_FLAG'] = np.where(outSample_df.FNMA_LN.isna(),0,1)
outSample_clean=outSample_df[["GROSMTHLYINCMAMT","BRWR_TOT_DTI_RTO_PCT","LN_APPL_INT_RT","LN_CLCD_LN_TO_VAL_RTO_PCT","PROP_APPRD_VAL_AMT","UWRG_LN_APPL_AMT","SUBCOUNT","UPBDIFF","CASE_FICO_NO","LN_COMB_LN_TO_VAL_RTO_PCT","LN_MTRY_TERM_MTHS_NO","LN_BRWR_CNT","NOMTHSRESVS","MTHLYHSNGEXP","TOTMNTHLYHSNGEXP","ELGCHG","TRIAL_LENDER","DU_ELGB_IND","PIW_OFR_IND","FRSTTMHMBYRIND","HIGHBAL","LN_AMRT_TYP_CD","LN_GUAR_TYP_CD","LN_PURPS_TYP_DESC","PRODFLG","PROP_USG_TYP_ID","PROP_CATG_TYP_ID","LN_RPTG_TYP_DESC","ACQ_FLAG"]].dropna()
outSample_pred_prob=model.predict_proba(outSample_clean)


/home/k2uxam/mypython/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (62,67,69,71,73,79,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
# calculate AUC for out-of sample
outSample_survival_ratio=[x[1] for x in outSample_pred_prob]
outSample_auc_score = metrics.roc_auc_score(outSample_clean.ACQ_FLAG,outSample_survival_ratio)
outSample_auc_score

0.7621749797463062

In [24]:
#create synthetic profile

pull_through=pd.read_csv('/home/k2uxam/testing_folder/pull_through_exc_prequals.csv')
pull_through_rate=pull_through.x

In [25]:
pull_through_rate

0     0.001236
1     0.001552
2     0.002312
3     0.003314
4     0.004580
5     0.007814
6     0.014010
7     0.032095
8     0.078973
9     0.131621
10    0.066083
11    0.002439
Name: x, dtype: float64

In [26]:
file_sort_name

['du_apps_201808.csv',
 'du_apps_201809.csv',
 'du_apps_201810.csv',
 'du_apps_201811.csv',
 'du_apps_201812.csv',
 'du_apps_201901.csv',
 'du_apps_201902.csv',
 'du_apps_201903.csv',
 'du_apps_201904.csv',
 'du_apps_201905.csv',
 'du_apps_201906.csv',
 'du_apps_201907.csv',
 'du_apps_201908.csv']

In [27]:
synthetic_list=[]
for i in range(0,12):
    du_monthly_data=du_files[i]
    du_monthly_data = du_monthly_data[(du_monthly_data['LAST_CASEFILE_INSTID'] == 1) & 
         (~(du_monthly_data['LN_RPTG_TYP_DESC'].str.contains('Federal Housing Administration') | du_monthly_data['LN_RPTG_TYP_DESC'].str.contains('Veterans Affairs'))) &
        (du_monthly_data['REFI_PLUS_IND'] == 0 ) & (du_monthly_data['CASE_FICO_NO'] >=620) ]
    du_monthly_data.dropna(subset=['BRWR_TOT_DTI_RTO_PCT'])
    du_monthly_data['ACQ_FLAG'] = np.where(du_monthly_data.FNMA_LN.isna(),0,1)
    du_monthly_final=du_monthly_data[["GROSMTHLYINCMAMT","BRWR_TOT_DTI_RTO_PCT","LN_APPL_INT_RT","LN_CLCD_LN_TO_VAL_RTO_PCT","PROP_APPRD_VAL_AMT","UWRG_LN_APPL_AMT","SUBCOUNT","UPBDIFF","CASE_FICO_NO","LN_COMB_LN_TO_VAL_RTO_PCT","LN_MTRY_TERM_MTHS_NO","LN_BRWR_CNT","NOMTHSRESVS","MTHLYHSNGEXP","TOTMNTHLYHSNGEXP","ELGCHG","TRIAL_LENDER","DU_ELGB_IND","PIW_OFR_IND","FRSTTMHMBYRIND","HIGHBAL","LN_AMRT_TYP_CD","LN_GUAR_TYP_CD","LN_PURPS_TYP_DESC","PRODFLG","PROP_USG_TYP_ID","PROP_CATG_TYP_ID","LN_RPTG_TYP_DESC","ACQ_FLAG"]].dropna()
    print('iteration')
    print(du_monthly_final.shape)
    monthly_syn_data=du_monthly_final.sample(frac=(pull_through_rate[(i)]))
    print(monthly_syn_data.shape)
    synthetic_list.append(monthly_syn_data)

/home/k2uxam/mypython/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


iteration
(686162, 29)
(848, 29)
iteration
(557795, 29)
(866, 29)
iteration
(610927, 29)
(1413, 29)
iteration
(503813, 29)
(1670, 29)
iteration
(429644, 29)
(1968, 29)
iteration
(637794, 29)
(4984, 29)
iteration
(622479, 29)
(8721, 29)
iteration
(769286, 29)
(24690, 29)
iteration
(831570, 29)
(65672, 29)
iteration
(807598, 29)
(106297, 29)
iteration
(842095, 29)
(55648, 29)
iteration
(878865, 29)
(2143, 29)


In [28]:
synthetic_df=pd.concat(synthetic_list)

In [29]:
synthetic_df.shape #figure out the issue here

(274920, 29)

In [30]:
#synthetic_df.to_csv('/home/k2uxam/testing_folder/catboost_synthetic_201908_exc_prequal.csv')
